In [1]:
import sys
from pprint import pprint
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent))

from tg.grammar_ru.common import Separator
import pandas as pd
pd.options.display.max_rows = 10

#### Попробуем построить модель, которая будет предсказывать род слова по левому контексту.


Род слова будем определять по pymorphy.

Насколько он хорош?

In [2]:
s="""
3. мы вы они я ты он она оно.
0. Около черного здания.
2. Около черного автомобиля.
3. Около черного окна.
4. По упавшему фонарю.
1. У стены стоит черный рояль.
2. На ужин мы приготовили жареный картофель.
3. По обеим сторонам дороги росли высокие деревья.
4. Друг пригласил меня на чашечку черного кофе.
5. Для прослойки торта можно использовать яблочное повидло.
6. В зале сидит компетентное жюри.
7. Ребенок с удовольствием есть картофельное пюре.
8. В праздники в доме обычно варили вкусный студень.
9. Я долго греб веслами, отчего на руке вздулась огромная мозоль.
10. Мама чисто убрала в комнате, на окна повесила красивый тюль.
11. Лицо дамы скрывала густая вуаль.
12. Отличник гордился своим табелем.
13. Машина ехала по асфальтированному шоссе.
14. На озере плавал черный лебедь.
15. Голову моют шампунем.
16. Я не нашла в магазинах ни апельсинов, ни мандаринов, ни баклажанов, зато купила 5 килограммов помидоров и килограмм яблок.
17. Солнечный Сочи – мой любимый город.
18. Иван Андреевич Крылов написал много басен.
19. Я встала, сняла ребенка с колен.
20. В поход мы взяли две пары сапог, несколько пар шерстяных чулок и носков.
21. В МГУ работают известные профессора.
22. В нашем городе много грузин, туркмен, узбеков, цыган, армян.
23. Осенью всегда бывает много свадеб.
24. Мы с мамой испекли пирожки с повидлом.
25. Гринев, как он сам говорил, был недорослем.
26. Золушка так торопилась, что потеряла одну туфлю.
27. Мы выехали на просторную авеню.
28. Жестокий юноша не слушал мольбы своей старой матери.
29. В этом ей должны были помочь две подруги.
30. Катя научила меня танцевать старинное танго."""


sentences = [x.split('. ') [1]  for x in s.split('.\n')]
text = ''.join([x + '. ' for x in sentences])
db = Separator.build_bundle(text)

orig = db.data_frames['src']



In [3]:
from tg.grammar_ru.ml.features import PyMorphyFeaturizer, SlovnetFeaturizer
import pymorphy2


pm_an = pymorphy2.MorphAnalyzer()
pmf = PyMorphyFeaturizer()
svf = SlovnetFeaturizer()
svf.featurize(db)
pmf.featurize(db)
df = db.data_frames['pymorphy']
df['word']=orig.word
pm = db.data_frames['pymorphy']


  0%|          | 0/279 [00:00<?, ?it/s]

In [4]:
import pymorphy2
an = pymorphy2.MorphAnalyzer()

In [16]:
an.parse('мы')

[Parse(word='мы', tag=OpencorporaTag('NPRO,1per plur,nomn'), normal_form='мы', score=1.0, methods_stack=((DictionaryAnalyzer(), 'мы', 2072, 0),))]

In [11]:
an.parse('мы')[0].tag.gender

In [32]:
from itertools import groupby

for score, v in groupby([(t.score, t.tag.gender) for t in pm_an.parse('черного')],
                                         lambda x: x[1]):
    # print(list(v))
    print()

0.428571
0.571428


In [5]:
pm[['gender', 'word']].head(10)

,gender,word
word_id,,
0,NaN,мы
1,NaN,вы
2,NaN,они
3,NaN,я
4,NaN,ты
5,masc,он
6,femn,она
7,neut,оно
8,NaN,.


In [6]:
pd.options.display.max_rows = 999
# pm[['word','gender', 'POS']][pm.POS.isin(["VERB"]) ]

* глаголы в множественном числе - Nan #приготовили

* Местоимения я, мы, вы, они, ты - Nan

* прилагательные во множественном ч. - Nan # ВЫСОКИЕ деревья

* когда форма прилагательного в ед.ч. не позволяет определить однозначно, 
ответы для существительного и прилагательного могут различаться

	* neut # на чашечку ЧЕРНОГО кофе. Около ЧЕРНОГО автомобиля. Около ЧЕРНОГО окна.
    
* Аналогично с причастием
	* masc # по АСФАЛЬТИРОВАННОМУ шоссе. По УПАВШЕМУ фонарю.

In [7]:
toc = pd.read_parquet('D:/1Education/grammar_ru/data-cache/bundles/grammatical_gender/prepare/raw/toc.parquet')
toc

,filename,timestamp,part_index,token_count,character_count,ordinal,max_id
file_id,,,,,,,
8103f6a8-5b17-4373-886e-364ea335b33b,,2022-11-11 15:55:06.907584,0,49995,279103,0,49995
42d3fe88-858b-4623-ad22-56fbcfe86ee0,,2022-11-11 15:55:33.162611,1,49982,278941,1,99977
71941e5a-3ce6-4645-9cc6-53d33ce4486a,,2022-11-11 15:56:03.309334,2,49988,280243,2,149965
b46a3224-191b-443a-b162-6de9272982c3,,2022-11-11 15:56:38.863736,3,49996,283280,3,199971
7d01fd7e-357a-4df5-8695-361efb76b27c,,2022-11-11 15:57:09.712439,4,49987,275535,4,249958
770f1aad-eb19-40d6-b3b8-c5131ed7f6e8,,2022-11-11 15:57:36.786415,5,49963,277275,5,299921
857bde8f-95c0-44e0-b141-0fbeeee4d0d8,,2022-11-11 15:58:00.583471,6,49996,275083,6,349917
7a54cbe5-266e-474c-a1f0-0704f1aef5c0,,2022-11-11 15:58:30.926176,7,49986,276963,7,399913
fd500f10-855d-46af-bea7-d916a5d64b0c,,2022-11-11 15:58:56.888336,8,49995,275191,8,449908


In [8]:
speech_part_labels = ['NOUN', 'ADJF', 'ADJS', 'VERB', 'PRTF', 'PRTS']
gender_labels = {g: i for i, g in enumerate(['masc', 'femn', 'neut', 'nan'])}
gender_labels

{'masc': 0, 'femn': 1, 'neut': 2, 'nan': 3}

In [9]:
df = pd.read_parquet('D:/1Education/grammar_ru/data-cache/bundles/grammatical_gender/prepare/raw/src/8103f6a8-5b17-4373-886e-364ea335b33b.parquet')
df.head(5)

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,file_id,corpus_id,label,is_target,original_word_id,original_sentence_id,original_paragraph_id,updated
0,0,0,0,0,1,Штабс-капитан,ru,13,c35847f3-2e69-464d-b14f-e271a1cd02cb,lenta.base.zip,0,True,0,0,0,False
1,1,0,1,0,0,П,ru,1,c35847f3-2e69-464d-b14f-e271a1cd02cb,lenta.base.zip,1,True,1,0,0,False
2,2,0,2,0,1,.,punct,1,c35847f3-2e69-464d-b14f-e271a1cd02cb,lenta.base.zip,-1,False,2,0,0,False
3,3,0,3,0,0,Н,ru,1,c35847f3-2e69-464d-b14f-e271a1cd02cb,lenta.base.zip,1,True,3,0,0,False
4,4,0,4,0,1,.,punct,1,c35847f3-2e69-464d-b14f-e271a1cd02cb,lenta.base.zip,-1,False,4,0,0,False


Тикеты на неделю:

*  ☑ Построить батч. Запустить нейросеть.

* ☑ Посмотреть ContextuallyAssemblyPoint, он умеет выдавать нейросеть. ого.

* ☑ pymorphy возвращает несколько вариантов. В бандле надо это учитывать

* проверить что возвращает pymorphy на Около ЧЕРНОГО автомобиля. Около ЧЕРНОГО окна.

* Попробовать левый контекст./ оба контекста.

* Как сделать маленький батч

* Запустить на своей машине на toy

* прикрутить метрику
